In [0]:
import pyspark.sql.functions as F

In [0]:
palabras_prohibidas = ["caca", "culo", "pedo", "pis", "mierda"]
regex_palabras_prohibidas = r"|".join(palabras_prohibidas)
regex_particionado = r"[ ,.;:_+()¿?¡!-]+"

In [0]:
datos_bruto=spark.table("bronze.tweets")
display(datos_bruto.head(5))

tweet,fecha
Preparando el examen de física de recuperación #MierdaDeVerano #StudyHard,2025-12-16T15:23:53.790595Z
Nada como un buen libro en la playa #Relax #GoodVibes,2025-12-16T15:23:53.790595Z
En la playa con amigos #SummerLove#GoodVibes,2025-12-16T15:23:53.790595Z
Disfrutando del sol #SummerLove #BeachTime,2025-12-16T15:23:53.790595Z


In [0]:
# Vamos a procesar los datos con spark.sql
# Ciertas operaciones (las que pueda hacer con SQL estandar) serán mucho más sencillas con esta librería (spark)
# Por contra, operaciones más complejas, serán más sencillas con la librería sparl.core (sc)
hashtags = (
    datos_bruto
    #.map( lambda tweet: tweet.replace("#", " #") ) \
        .withColumn("tweet_normalizado", F.regexp_replace("tweet", "#", " #") )
    #.flatMap(     lambda tweet: re.split(regex_particionado, tweet) ) \
        # La parte del map
        .withColumn("tweeet_tokenizado", F.split("tweet_normalizado", regex_particionado) )
        # La parte del flatten
        .withColumn("tokens", F.explode("tweeet_tokenizado"))
    # Como se nos generan muchas columnas que ya no son necesarias, podemos eliminarlas
        .drop("tweet", "tweet_normalizado", "tweeet_tokenizado")
        #.select("fecha", "tokens")
    #.filter(      lambda palabra: palabra[0] == "#" ) \
        .filter(F.col("tokens").startswith("#"))
    #.map(         lambda hashtag: hashtag.lower() ) \
        .withColumn("tokens", F.lower("tokens"))
    #.map(         lambda hashtag: hashtag[1:] ) \
        #.withColumn("tokens", F.regexp_replace("tokens", "#", ""))
        .withColumn("tokens", F.expr("substring(tokens, 2, length(tokens))"))
    #.filter(      lambda hashtag: len([palabra_prohibida for palabra_prohibida in palabras_prohibidas if 
    # palabra_prohibida in hashtag] ) == 0 ) \
        .filter(~F.col("tokens").rlike(regex_palabras_prohibidas ))
)
display(hashtags)

fecha,tokens
2025-12-16T15:23:53.790595Z,studyhard
2025-12-16T15:23:53.790595Z,relax
2025-12-16T15:23:53.790595Z,goodvibes
2025-12-16T15:23:53.790595Z,summerlove
2025-12-16T15:23:53.790595Z,goodvibes
2025-12-16T15:23:53.790595Z,summerlove
2025-12-16T15:23:53.790595Z,beachtime


In [0]:
# Nos aseguramos que exista laa capa silver
spark.sql("CREATE DATABASE IF NOT EXISTS silver")
# Guardamos la tabla en capa silver
(hashtags
    .write
    .format("delta")
    .mode("append")
    .saveAsTable("silver.hashtags")
)